In [1]:
import numpy as np
import pandas as pd
import os
import time
import sys
from data import data_summary
from data import data_waterfall

%load_ext autoreload
%autoreload 2

In [2]:
df_dev = pd.read_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/raw_tables/af_member_base_before_exc.parquet')

In [31]:
df_dev['no_fico_flag']= np.where((df_dev.applicant_fico_score.isnull()) | (df_dev.applicant_fico_score>850),1,0)

In [32]:
def drop_bureau_no_match(df):
    return df[~(df['no_match_flag'] == 1)]

def drop_bureau_no_hit(df):
    return df[~(df['no_fico_flag'] == 1)]

In [33]:
df_dev['weight'] = np.where(df_dev['coborrower_indicator'] == 'N', 1, 0.5)
df_dev['coborrower_indicator'].value_counts()

N    3232823
Y     312716
Name: coborrower_indicator, dtype: int64

In [34]:
fns = [drop_bureau_no_match,
       drop_bureau_no_hit]
wf = data_waterfall.DataWaterfallReport(df_dev, fns,'weight')

In [35]:
df_waterfall = wf.get_report(True)
df_waterfall

,# Excluded,# Remain,% Excluded
Starting Population,,3389181,
Drop Bureau No Match,175864,3213316,5.19%
Drop Bureau No Hit,111873,3101443,3.30%


In [36]:
fns = [drop_bureau_no_hit]
wf = data_waterfall.DataWaterfallReport(df_dev[df_dev.no_match_flag==1], fns,'weight')

df_waterfall = wf.get_report(True)
df_waterfall

,# Excluded,# Remain,% Excluded
Starting Population,,175864,
Drop Bureau No Hit,59368,116496,33.76%


In [37]:
df_dev['no_fico_flag'].mean()

0.05057876954674592

In [39]:
1-.3376

0.6624